In [ ]:
import tensorflow as tensorflow
from tensorflow.keras.layers import Input, Conv2D, Activation, Add

def residual_block(input_tensor, num_filters,strides=1):
    """
    入力と出力のサイズが変わらない、基本的な残差ブロック
    残差：Residual
    Output = f(Input)+Input
        ここでFは2回の畳み込みとReLU活性化

    Skip Connection
        残差分だけを学習することで、層が深くなっても勾配損失問題を防ぐ
    
    Conv
        畳み込みをすると通常は画面の端っこが削れて小さくなるが、padding='same'を指定して周りに余白をつける
        そうすると入力と出力のサイズが変わらなくなる

    Add()([x,shortcut])
        KerasのFunctional APIの書き方
            ➀足し算マシーン(Addレイヤー)の生成、ここでAddは単純に足し算をするだけなので()の中に複雑な設定が必要ない
            ➁足し算マシーンに[x,shortcut]というリストの形状にして足し算を実行する

    Strides
        畳み込みの移動幅を指定するパラメータ
        1なら通常通り1pxずつ、2なら2pxずつ移動する(1個飛ばし)ので、出力サイズが半分になる

    num_filters
        各num_filterは畳み込み層で検知した特徴マップを持っている
        例えば浅い層のフィルターではエッジ検出、深い層ではより複雑なパターンを検出する
            エッジ：隣り合う画素(ピクセル)が急激に変化している部分
    """
    shortcut = input_tensor

    # 畳み込み一回目
    x = Conv2D(num_filters, (3, 3), padding='same',strides=strides)(input_tensor)
    x=BatchNormalization()(x)
    x = Activation('relu')(x)


    x = Conv2D(num_filters, (3, 3), padding='same')(x) #畳み込み二回目


    if strides > 1 or input_tensor.shape[-1] != num_filters:
        """
        サイズが変わる、またはチャンネル数が変わるとき
        .shape[-1]は最後の要素、つまりchannnel
        """
        shortcut = Conv2D(num_filters, (1, 1), padding='same',strides=strides)(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense

def build_simple_resnet(input_shape=(128, 128, 3), num_classes=10):
    """
    ResNetの簡易モデルの作成
        画像分類タスクでは、画像の特徴の身を知りたいので畳み込み層のエンコードしか必要ない
        セグメンテーションは位置情報も必要なのでdecodeも必要になる
        ResNet18の18は層の数、50や152などがある

    Projection(射影)
            ➀入力の特徴マップのサイズを変換する
            ➁フィルター数(channel数)を変更する

    GlobalAveragePooling2D
        Strides=1は画像サイズを変えないまま特徴をしっかり抽出する
        
    """
    inputs = Input(input_shape)
    x = residual_block(inputs, num_filters=64, strides=1)

    x = residual_block(x, num_filters=64, strides=1)   #サイズを変えないまま特徴をしっかり見極める
    x = residual_block(x, num_filters=64, strides=1) 
    
    x = residual_block(x, num_filters=128, strides=2)   #サイズを半分にする(Projection Block)
    x = residual_block(x, num_filters=128, strides=1) 
    
    # --- 出口 (Output Layers) ---
    # Q4: 画像全体を平均して1次元のベクトルにする
    x = GlobalAveragePooling2D()(x) 
    
    # 最終判定 (10クラス分類なので softmax)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs, outputs)